## Libraries

In [1]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from linearmodels.panel.model import PanelOLS
from statsmodels.regression.linear_model import OLS

## Checking for Causality

In [2]:
df = pd.read_csv('311_borough_weather.csv')
df['Y-M-d'] = pd.to_datetime(df['Y-M-d'])

A helper function to decompose a series of values into 3 signals - trend, seasonal, residual

In [3]:
def create_trend_seasonality_error(data, cols): 
    for col in cols:
        res = sm.tsa.seasonal_decompose(data[col], period=12, extrapolate_trend='freq')
        data[col+'_trend'] = res.trend
        data[col+'_seasonal'] = res.seasonal
        data[col+'_error'] = res.resid      
    return data

Label encoding the different boroughs with a value

In [4]:
borough_mapper = {}

for index, value in enumerate(df['Borough'].unique()):
    borough_mapper[value] = index

df['Borough'] = df['Borough'].map(borough_mapper)

In [5]:
df

,Y-M-d,Borough,Count,Date,DailyMeanTemp,DailyTempStd
0,2016-01-01,0,689,2016-01-01,41.250000,0.758068
1,2016-01-01,1,858,2016-01-01,41.250000,0.758068
2,2016-01-01,2,558,2016-01-01,41.250000,0.758068
3,2016-01-01,3,606,2016-01-01,41.250000,0.758068
4,2016-01-01,4,69,2016-01-01,41.250000,0.758068
...,...,...,...,...,...,...
6571,2018-12-31,1,1382,2018-12-31,47.500135,12.899035
6572,2018-12-31,2,652,2018-12-31,47.500135,12.899035
6573,2018-12-31,3,939,2018-12-31,47.500135,12.899035
6574,2018-12-31,4,213,2018-12-31,47.500135,12.899035


Aggregating the data on a daily level

In [6]:
df_weather_week = df.groupby('Y-M-d').agg({'DailyMeanTemp': 'unique', 'DailyTempStd': 'unique'})

In [7]:
df_weather_week = df_weather_week.reset_index()
df_weather_week['DailyMeanTemp'] = df_weather_week['DailyMeanTemp'].apply(lambda x: x[0])
df_weather_week['DailyTempStd'] = df_weather_week['DailyTempStd'].apply(lambda x: x[0])

Calculating the lagged features from AR term - 7 and MA featurs - 2, for each borough

In [8]:
ets_train_data_master = pd.DataFrame()
for i in df['Borough'].unique():
    df_boro = df[df['Borough'] == i]
    df_boro = df_boro.reset_index(drop = True)
    df_boro.sort_values(by = 'Y-M-d', inplace = True)

    lagged_train_data = df_boro.copy()
    trailing_window_size = 7
    df_boro_lags = pd.DataFrame()

    for window in range(1, trailing_window_size + 1):
        shifted = df.loc[:, 'Count'].shift(window)
        df_boro_lags[f'lag_{window}'] = shifted
        
    df_boro_lags['Y-M-d'] = df_boro['Y-M-d']

    df_boro_lags = pd.merge(df_boro, df_boro_lags, how = 'inner', on = 'Y-M-d')

    ets_train_data = create_trend_seasonality_error(df_boro_lags.copy(), ['Count'])
    
    ets_train_data['MA_1'] = ets_train_data['Count'].rolling(1).mean().shift(1)
    ets_train_data['MA_2'] = ets_train_data['Count'].rolling(2).mean().shift(2)
    
    ets_train_data = ets_train_data.dropna()
    ets_train_data_master = pd.concat([ets_train_data_master, ets_train_data])

The data being in a panel format, we perform a Panel OLS to account for time dummies and fixed effects. <br>
After having accounted for both, we get a p-value for DailyMeanTemp < 0.05 <br>
This implies that temperature does play a role in predicting the number of calls

In [9]:
ets_train_data_master['Date'] = pd.to_datetime(ets_train_data_master['Date'])
ets_train_data_master['Year'] = ets_train_data_master['Date'].apply(lambda x: x.year)
ets_train_data_master['Month'] = ets_train_data_master['Date'].apply(lambda x: x.month)
ets_train_data_master['Day'] = ets_train_data_master['Date'].apply(lambda x: x.day)

del ets_train_data_master['Date']

In [10]:
ets_train_data_master = pd.get_dummies(ets_train_data_master, columns=['Year', 'Month', 'Day'], drop_first=True)

The effect of temperature can hence be said is significant

In [11]:
ets_train_data_master = ets_train_data_master.set_index(['Borough', 'Y-M-d'])
cols = list(ets_train_data_master.columns)

cols.remove('Count')
cols.remove('Count_trend')
cols.remove('Count_seasonal')
cols.remove('Count_error')

mod = PanelOLS(ets_train_data_master['Count'], ets_train_data_master[cols],
            entity_effects=True, drop_absorbed=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:                  Count   R-squared:                        0.3959
Estimator:                   PanelOLS   R-squared (Between):              0.6986
No. Observations:                6534   R-squared (Within):               0.3959
Date:                Sun, Dec 25 2022   R-squared (Overall):              0.6856
Time:                        07:03:48   Log-likelihood                -4.173e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      78.575
Entities:                           6   P-value                           0.0000
Avg Obs:                       1089.0   Distribution:                 F(54,6474)
Min Obs:                       1089.0                                           
Max Obs:                       1089.0   F-statistic (robust):         -1.638e+17
                            